### 🐁 AmadeusGPT Quickstart

---


#### Notebook Overview:
1. Bootstrap the Jupyter interpreter 🛠️
2. Load secrets & config 🔑
3. Project initialization 📁
4. Run a demo query 🎬
5. Appendix 📚

#### 1 · Bootstrap the Jupyter interpreter 🛠️
-  Run the below cell *once* per fresh clone or when dependencies change
-  ⚠️ Select a 3.10 Python environment before you run the below code (if you already have the AmadeusGPT Jupyter kernel select that and skip to [Jump to “Load secrets & config”](#load-secrets-config)
)
<a id="bootstrap-jupyter-interpreter"></a>

In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet -e .[dev]            # editable install → reads pyproject.toml
%pip install --quiet ipykernel       # needed for kernel registration
%matplotlib inline

Register the interpreter as a named Jupyter kernel - unless you already have access to the AmadeusGPT (venv)

In [ ]:
!python -m ipykernel install --user --name amadeus-gpt-env --display-name "AmadeusGPT (venv)"

print("✅  Bootstrapping done – now **Restart** the kernel (Kernel ▶ Restart) and select “AmadeusGPT (venv)” in the top-right kernel picker.")

<a id="load-secrets-config"></a>
#### 2 · Load secrets & config 🔑


In [2]:
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
load_dotenv(env_path, override=True)
print("🔐  Environment loaded from:", env_path or "❌  .env file not found. Create one and add your OPENAI_API_KEY='api_key' to complete this step")


🔐  Environment loaded from: c:\Users\hugos\Desktop\AmadeusGPT\.env


#### 3 · Project initialisation 📁
You can 

In [ ]:
import os
from amadeusgpt import AMADEUS, create_project
from amadeusgpt.utils import parse_result

# ── resolve repo root dynamically ──
ROOT = os.getcwd()

DATA_DIR   = os.path.join(ROOT, "data")
RESULT_DIR = os.path.join(ROOT, "results")

project_cfg = create_project(
    data_folder   = DATA_DIR,
    result_folder = RESULT_DIR,
    **{
        "data_info.video_suffix": ".avi",
        "keypoint_info.use_3d"  : True,
    }
)

amadeus = AMADEUS(project_cfg)
print("✅  Project ready:", project_cfg)


Project created at c:\Users\hugos\Desktop\AmadeusGPT\results. Results will be saved to c:\Users\hugos\Desktop\AmadeusGPT\results
The project will load video files (*.avi) and optionally keypoint files from c:\Users\hugos\Desktop\AmadeusGPT\data
A copy of the project config file is saved at c:\Users\hugos\Desktop\AmadeusGPT\results\config.yaml
{'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data',
               'result_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\results',
               'video_suffix': '.avi'},
 'keypoint_info': {'include_confidence': False, 'use_3d': True},
 'llm_info': {'gpt_model': 'gpt-4o',
              'keep_last_n_messages': 2,
              'max_tokens': 4096,
              'temperature': 0.0},
 'object_info': {'load_objects_from_disk': False, 'use_grid_objects': False},
 'video_info': {'scene_frame_number': 1}}
✅  Project ready: {'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data', 'result_folder': 'c:\\Users\\

#### 4 · Run a demo query 🎬


### Appendix 📚

---

In [ ]:
query = "Define <|rearing|> as a behavior where the mouse's head, including its snout, right, left ear, and neck base, when their height is averaged together, is at least two and a half times as high as the height of the averaged tail base and body tail: the body midpoint should be between these two averaged values: if it isn't assume the animal is doing another behavior. Get the frames where the mouse is rearing and plot them as a rectangular timeline plot with the x-axis being the frame numbers and a purple color scheme showing the frames during which the mouse was rearing. In a second plot, show the average height of the mouse's head, snout, and ears over time, with a red line indicating the threshold for rearing behavior."
msg   = amadeus.step(query)
parse_result(amadeus, msg)

query2 = "Save the results of the rearing behavior analysis to a file named 'rearing_behavior_analysis.txt' in the 'results' folder. Ensure that the file includes the specific frame ranges where the mouse was rearing, the average height of the mouse's head, snout, and ears per frame, and any relevant statistics or observations from the analysis. The file should be formatted clearly for easy reading and interpretation."
msg2 = amadeus.step(query2)

parse_result(amadeus, msg2)
print("✅  Query executed successfully!")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


doing active forgetting


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.4347 $
current input tokens 3961
current accumulated tokens 63440


To address your request, I will write a function that identifies the frames where the mouse is rearing based on the defined criteria. Then, I will create two plots: 

1. A rectangular timeline plot showing the frames during which the mouse was rearing, using a purple color scheme.
2. A plot showing the average height of the mouse's head, snout, and ears over time, with a red line indicating the threshold for rearing behavior.

Here's the implementation:

```python
def plot_rearing_behavior(identifier):
    '''
    Analyzes and plots the rearing behavior of a mouse.

    Parameters
    ----------
    identifier: Identifier
        Contains information about the video, keypoint, and config.
    '''
    import numpy as np
    import matplotlib.pyplot as plt

    # Create an instance of AnimalBehaviorAnalysis
    analysis = create_analysis(identifier)

    # Get keypoints and their names
    keypoints = analysis.get_keypoints()  # shape: (n_frames, n_individuals, n_kpts, n_dims)
    keypoint_names = analysis.get_keypoint_names()

    # Extract indices for the required keypoints
    snout_idx = keypoint_names.index('Snout')
    left_ear_idx = keypoint_names.index('Left ear')
    right_ear_idx = keypoint_names.index('Right ear')
    neck_base_idx = keypoint_names.index('Neck base')
    tail_base_idx = keypoint_names.index('Tail base')
    body_tail_idx = keypoint_names.index('Body-tail midpoint')
    body_midpoint_idx = keypoint_names.index('Body midpoint')

    # Extract the z-coordinates (height) for the relevant keypoints
    head_heights = keypoints[:, 0, [snout_idx, left_ear_idx, right_ear_idx, neck_base_idx], 2]  # shape: (n_frames, 4)
    tail_heights = keypoints[:, 0, [tail_base_idx, body_tail_idx], 2]  # shape: (n_frames, 2)
    body_midpoint_height = keypoints[:, 0, body_midpoint_idx, 2]  # shape: (n_frames,)

    # Calculate average heights
    avg_head_height = np.mean(head_heights, axis=1)  # shape: (n_frames,)
    avg_tail_height = np.mean(tail_heights, axis=1)  # shape: (n_frames,)

    # Determine rearing frames
    rearing_frames = (avg_head_height >= 2.5 * avg_tail_height) & (body_midpoint_height > avg_tail_height) & (body_midpoint_height < avg_head_height)

    # Plotting
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    # Plot 1: Rectangular timeline plot
    axs[0].bar(range(len(rearing_frames)), rearing_frames, color='purple', width=1.0)
    axs[0].set_title('Rearing Behavior Timeline')
    axs[0].set_xlabel('Frame Number')
    axs[0].set_ylabel('Rearing (1 = Yes, 0 = No)')
    axs[0].set_xlim(0, len(rearing_frames))

    # Plot 2: Average head height over time
    axs[1].plot(avg_head_height, label='Average Head Height', color='blue')
    axs[1].axhline(y=2.5 * np.mean(avg_tail_height), color='red', linestyle='--', label='Rearing Threshold')
    axs[1].set_title('Average Head Height Over Time')
    axs[1].set_xlabel('Frame Number')
    axs[1].set_ylabel('Height')
    axs[1].legend()
    axs[1].set_xlim(0, len(avg_head_height))

    plt.tight_layout()
    return fig, axs
```

This function will analyze the rearing behavior based on the specified criteria and generate two plots. The first plot is a timeline showing when the mouse is rearing, and the second plot shows the average head height over time with a threshold line for rearing behavior.

...

doing active forgetting


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.473 $
current input tokens 3943
current accumulated tokens 68621


To save the results of the rearing behavior analysis to a file, I will modify the function to include writing the analysis results to a text file named 'rearing_behavior_analysis.txt' in the 'results' folder. The file will include the specific time ranges when the mouse was rearing, the average height of the mouse's head, snout, and ears per frame, and any relevant statistics or observations.

Here's the updated implementation:

```python
def plot_and_save_rearing_behavior(identifier):
    '''
    Analyzes, plots, and saves the rearing behavior of a mouse.

    Parameters
    ----------
    identifier: Identifier
        Contains information about the video, keypoint, and config.
    '''
    import numpy as np
    import matplotlib.pyplot as plt
    import os

    # Create an instance of AnimalBehaviorAnalysis
    analysis = create_analysis(identifier)

    # Get keypoints and their names
    keypoints = analysis.get_keypoints()  # shape: (n_frames, n_individuals, n_kpts, n_dims)
    keypoint_names = analysis.get_keypoint_names()

    # Extract indices for the required keypoints
    snout_idx = keypoint_names.index('Snout')
    left_ear_idx = keypoint_names.index('Left ear')
    right_ear_idx = keypoint_names.index('Right ear')
    neck_base_idx = keypoint_names.index('Neck base')
    tail_base_idx = keypoint_names.index('Tail base')
    body_tail_idx = keypoint_names.index('Body-tail midpoint')
    body_midpoint_idx = keypoint_names.index('Body midpoint')

    # Extract the z-coordinates (height) for the relevant keypoints
    head_heights = keypoints[:, 0, [snout_idx, left_ear_idx, right_ear_idx, neck_base_idx], 2]  # shape: (n_frames, 4)
    tail_heights = keypoints[:, 0, [tail_base_idx, body_tail_idx], 2]  # shape: (n_frames, 2)
    body_midpoint_height = keypoints[:, 0, body_midpoint_idx, 2]  # shape: (n_frames,)

    # Calculate average heights
    avg_head_height = np.mean(head_heights, axis=1)  # shape: (n_frames,)
    avg_tail_height = np.mean(tail_heights, axis=1)  # shape: (n_frames,)

    # Determine rearing frames
    rearing_frames = (avg_head_height >= 2.5 * avg_tail_height) & (body_midpoint_height > avg_tail_height) & (body_midpoint_height < avg_head_height)

    # Get frame rate to convert frames to seconds
    frame_rate = analysis.get_frame_rate()

    # Calculate time ranges for rearing behavior
    rearing_time_ranges = []
    start_frame = None
    for i, is_rearing in enumerate(rearing_frames):
        if is_rearing and start_frame is None:
            start_frame = i
        elif not is_rearing and start_frame is not None:
            end_frame = i - 1
            rearing_time_ranges.append((start_frame / frame_rate, end_frame / frame_rate))
            start_frame = None
    if start_frame is not None:
        rearing_time_ranges.append((start_frame / frame_rate, len(rearing_frames) / frame_rate))

    # Save results to a file
    os.makedirs('results', exist_ok=True)
    with open('results/rearing_behavior_analysis.txt', 'w') as file:
        file.write("Rearing Behavior Analysis\n")
        file.write("=========================\n\n")
        file.write("Time Ranges (in seconds) when the mouse was rearing:\n")
        for start, end in rearing_time_ranges:
            file.write(f"Start: {start:.2f}s, End: {end:.2f}s\n")
        file.write("\nAverage Head Heights per Frame:\n")
        for i, height in enumerate(avg_head_height):
            file.write(f"Frame {i}: {height:.2f}\n")
        file.write("\nObservations:\n")
        file.write(f"Total Rearing Duration: {sum(end - start for start, end in rearing_time_ranges):.2f} seconds\n")
        file.write(f"Total Frames: {len(rearing_frames)}\n")
        file.write(f"Frame Rate: {frame_rate} fps\n")

    # Plotting
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    # Plot 1: Rectangular timeline plot
    axs[0].bar(range(len(rearing_frames)), rearing_frames, color='purple', width=1.0)
    axs[0].set_title('Rearing Behavior Timeline')
    axs[0].set_xlabel('Frame Number')
    axs[0].set_ylabel('Rearing (1 = Yes, 0 = No)')
    axs[0].set_xlim(0, len(rearing_frames))

    # Plot 2: Average head height over time
    axs[1].plot(avg_head_height, label='Average Head Height', color='blue')
    axs[1].axhline(y=2.5 * np.mean(avg_tail_height), color='red', linestyle='--', label='Rearing Threshold')
    axs[1].set_title('Average Head Height Over Time')
    axs[1].set_xlabel('Frame Number')
    axs[1].set_ylabel('Height')
    axs[1].legend()
    axs[1].set_xlim(0, len(avg_head_height))

    plt.tight_layout()
    return fig, axs
```

This function will analyze the rearing behavior, save the results to a text file, and generate the requested plots. The text file will include the time ranges when the mouse was rearing, the average head heights per frame, and relevant statistics.

✅  Query executed successfully!



#### 1. Updating code  
- After you’ve made changes to your package source, simply re-run the dependency‐install cell. [Re-install dependencies here.](#bootstrap-jupyter-interpreter)
#### 2. Troubleshooting

* **Kernel name not visible?**

  * Reload VS Code/Notebook window (e.g. ⌘⇧P → “Developer: Reload Window”).
  * Make sure you're selecting Jupyter Kernels not Python Environments to find `AmadeusGPT (venv)`
  * Ensure the workspace interpreter is set to your `.venv` (click the Python version in the status bar).

* **Version or Downloading Issues?**
  
  * Versioning errors occur if your Kernel is not Python version 3.10 (required for AmadeusGPT).
    *  Ensure that you have this version of Python installed and have selected it as the version for your environment
    *  You can check this in top right corner. If you see it's Python 3.10... that's not the issue.

* **“Package X not found” after reinstalling**

  * Confirm you’ve restarted the kernel (Kernel ▶ Restart).
  * Run `pip list` in a cell to verify that your package is installed in the active environment.

* **Still stuck?**

  * In a terminal, run:

    ```bash
    jupyter kernelspec list
    ```

    to verify that `amadeus-gpt-env` appears under your user kernels.
  * Stop & restart the Jupyter server
    (in VS Code: ⌘⇧P → “Jupyter: Stop Jupyter Server”, then run any cell again).